In [33]:
import os

# Replace with the actual path where your Mosaic folders are located
base_directory = "D:/BUET/Thesis/ArcGIS/Mosaiced Landsat"

In [34]:
# Loop through each Mosaic folder from 1988 to 2024
for year in range(1988, 2025):
    folder_name = f"Mosaic_{year}"
    folder_path = os.path.join(base_directory, folder_name)

    # Check if the folder exists
    if os.path.isdir(folder_path):
        print(f"\nFiles in {folder_name}:")

        # List all files in the folder
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".tif"):
                print(file_name)
    else:
        print(f"{folder_name} folder does not exist.")


Files in Mosaic_1988:
Mosaic_1988_B2.tif
Mosaic_1988_B3.tif
Mosaic_1988_B4.tif

Files in Mosaic_1989:
Mosaic_1989_B2.tif
Mosaic_1989_B3.tif
Mosaic_1989_B4.tif

Files in Mosaic_1990:
Mosaic-1990_B4.tif
Mosaic_1990_B2.tif
Mosaic_1990_B3.tif

Files in Mosaic_1991:
Mosaic_1991_B2.tif
Mosaic_1991_B3.tif
Mosaic_1991_B4.tif

Files in Mosaic_1992:
Mosaic_1992_B2.tif
Mosaic_1992_B3.tif
Mosaic_1992_B4.tif

Files in Mosaic_1993:
Mosaic_1993_B2.tif
Mosaic_1993_B3.tif
Mosaic_1993_B4.tif

Files in Mosaic_1994:
Mosaic_1994_B2.tif
Mosaic_1994_B3.tif
Mosaic_1994_B4.tif

Files in Mosaic_1995:
Mosaic_1995_B2.tif
Mosaic_1995_B3.tif
Mosaic_1995_B4.tif

Files in Mosaic_1996:
Mosaic_1996_B2.tif
Mosaic_1996_B3.tif
Mosaic_1996_B4.tif

Files in Mosaic_1997:
Mosaic_1997_B2.tif
Mosaic_1997_B3.tif
Mosaic_1997_B4.tif

Files in Mosaic_1998:
Mosaic_1998_B2.tif
Mosaic_1998_B3.tif
Mosaic_1998_B4.tif

Files in Mosaic_1999:
Mosaic_1999_B2.tif
Mosaic_1999_B3.tif
Mosaic_1999_B4.tif

Files in Mosaic_2000:
Mosaic_2000_B2.ti

In [35]:
import os
import numpy as np
import rasterio
from rasterio.enums import Resampling
from rasterio.vrt import WarpedVRT

In [36]:
# Base directory where Mosaic folders are located
base_directory = "D:/BUET/Thesis/ArcGIS/Mosaiced Landsat"
output_directory = "D:/BUET/Thesis/ArcGIS/NDVI2"  # Directory to save NDVI output files
os.makedirs(output_directory, exist_ok=True)

In [37]:
# Loop through each year from 1988 to 2024
for year in range(1988, 2025):
    folder_name = f"Mosaic_{year}"
    folder_path = os.path.join(base_directory, folder_name)
    
    if os.path.isdir(folder_path):
        # Define band file names based on the year
        if year <= 2013:
            band3_filename = f"Mosaic_{year}_B3.tif"
            band4_filename = f"Mosaic_{year}_B4.tif"
            equation = "(Band 4 - Band 3) / (Band 4 + Band 3)"
        else:
            band4_filename = f"Mosaic_{year}_B4.tif"
            band5_filename = f"Mosaic_{year}_B5.tif"
            equation = "(Band 5 - Band 4) / (Band 5 + Band 4)"

        # Paths for the required band files
        band3_path = os.path.join(folder_path, band3_filename) if year <= 2013 else None
        band4_path = os.path.join(folder_path, band4_filename)
        band5_path = os.path.join(folder_path, band5_filename) if year > 2013 else None

        # Check if necessary band files exist
        if (year <= 2013 and os.path.isfile(band3_path) and os.path.isfile(band4_path)) or \
           (year > 2013 and os.path.isfile(band4_path) and os.path.isfile(band5_path)):
            
            # Read band data and align if needed
            with rasterio.open(band4_path) as band4_src:
                band4 = band4_src.read(1)
                
                # For band3 (1988-2013) or band5 (2014-2024), warp to match dimensions of band4
                if year <= 2013:
                    with rasterio.open(band3_path) as band3_src:
                        with WarpedVRT(band3_src, width=band4_src.width, height=band4_src.height,
                                       transform=band4_src.transform, resampling=Resampling.nearest) as band3_vrt:
                            band3 = band3_vrt.read(1)
                    ndvi = (band4 - band3) / (band4 + band3)
                else:
                    with rasterio.open(band5_path) as band5_src:
                        with WarpedVRT(band5_src, width=band4_src.width, height=band4_src.height,
                                       transform=band4_src.transform, resampling=Resampling.nearest) as band5_vrt:
                            band5 = band5_vrt.read(1)
                    ndvi = (band5 - band4) / (band5 + band4)
                
                # Update metadata for output file
                profile = band4_src.profile
                profile.update(dtype=rasterio.float32, count=1)
                output_file_path = os.path.join(output_directory, f"NDVI_{year}.tif")

                # Save NDVI result as a new .tif file
                with rasterio.open(output_file_path, "w", **profile) as dst:
                    dst.write(ndvi.astype(rasterio.float32), 1)
                
                print(f"NDVI for {year} saved as {output_file_path} using {equation}")
        
        else:
            print(f"Missing necessary files for {year}. Skipping...")
    else:
        print(f"{folder_name} folder does not exist. Skipping...")

C:\Users\gg\AppData\Local\Temp\ipykernel_8604\1962984636.py:36: RuntimeWarning: overflow encountered in add
  ndvi = (band4 - band3) / (band4 + band3)


NDVI for 1988 saved as D:/BUET/Thesis/ArcGIS/NDVI2\NDVI_1988.tif using (Band 4 - Band 3) / (Band 4 + Band 3)


C:\Users\gg\AppData\Local\Temp\ipykernel_8604\1962984636.py:36: RuntimeWarning: invalid value encountered in divide
  ndvi = (band4 - band3) / (band4 + band3)


NDVI for 1989 saved as D:/BUET/Thesis/ArcGIS/NDVI2\NDVI_1989.tif using (Band 4 - Band 3) / (Band 4 + Band 3)
Missing necessary files for 1990. Skipping...
NDVI for 1991 saved as D:/BUET/Thesis/ArcGIS/NDVI2\NDVI_1991.tif using (Band 4 - Band 3) / (Band 4 + Band 3)
NDVI for 1992 saved as D:/BUET/Thesis/ArcGIS/NDVI2\NDVI_1992.tif using (Band 4 - Band 3) / (Band 4 + Band 3)
NDVI for 1993 saved as D:/BUET/Thesis/ArcGIS/NDVI2\NDVI_1993.tif using (Band 4 - Band 3) / (Band 4 + Band 3)
NDVI for 1994 saved as D:/BUET/Thesis/ArcGIS/NDVI2\NDVI_1994.tif using (Band 4 - Band 3) / (Band 4 + Band 3)
NDVI for 1995 saved as D:/BUET/Thesis/ArcGIS/NDVI2\NDVI_1995.tif using (Band 4 - Band 3) / (Band 4 + Band 3)
NDVI for 1996 saved as D:/BUET/Thesis/ArcGIS/NDVI2\NDVI_1996.tif using (Band 4 - Band 3) / (Band 4 + Band 3)
NDVI for 1997 saved as D:/BUET/Thesis/ArcGIS/NDVI2\NDVI_1997.tif using (Band 4 - Band 3) / (Band 4 + Band 3)
NDVI for 1998 saved as D:/BUET/Thesis/ArcGIS/NDVI2\NDVI_1998.tif using (Band 4 - B

C:\Users\gg\AppData\Local\Temp\ipykernel_8604\1962984636.py:42: RuntimeWarning: overflow encountered in add
  ndvi = (band5 - band4) / (band5 + band4)


NDVI for 2014 saved as D:/BUET/Thesis/ArcGIS/NDVI2\NDVI_2014.tif using (Band 5 - Band 4) / (Band 5 + Band 4)


C:\Users\gg\AppData\Local\Temp\ipykernel_8604\1962984636.py:42: RuntimeWarning: invalid value encountered in divide
  ndvi = (band5 - band4) / (band5 + band4)


NDVI for 2015 saved as D:/BUET/Thesis/ArcGIS/NDVI2\NDVI_2015.tif using (Band 5 - Band 4) / (Band 5 + Band 4)
NDVI for 2016 saved as D:/BUET/Thesis/ArcGIS/NDVI2\NDVI_2016.tif using (Band 5 - Band 4) / (Band 5 + Band 4)
NDVI for 2017 saved as D:/BUET/Thesis/ArcGIS/NDVI2\NDVI_2017.tif using (Band 5 - Band 4) / (Band 5 + Band 4)
NDVI for 2018 saved as D:/BUET/Thesis/ArcGIS/NDVI2\NDVI_2018.tif using (Band 5 - Band 4) / (Band 5 + Band 4)
NDVI for 2019 saved as D:/BUET/Thesis/ArcGIS/NDVI2\NDVI_2019.tif using (Band 5 - Band 4) / (Band 5 + Band 4)
NDVI for 2020 saved as D:/BUET/Thesis/ArcGIS/NDVI2\NDVI_2020.tif using (Band 5 - Band 4) / (Band 5 + Band 4)
NDVI for 2021 saved as D:/BUET/Thesis/ArcGIS/NDVI2\NDVI_2021.tif using (Band 5 - Band 4) / (Band 5 + Band 4)
NDVI for 2022 saved as D:/BUET/Thesis/ArcGIS/NDVI2\NDVI_2022.tif using (Band 5 - Band 4) / (Band 5 + Band 4)
NDVI for 2023 saved as D:/BUET/Thesis/ArcGIS/NDVI2\NDVI_2023.tif using (Band 5 - Band 4) / (Band 5 + Band 4)
NDVI for 2024 saved